In [1]:
import pandas as pd
from typing import Iterable, Literal, overload
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2 as cv
import numpy as np
import math
import os

In [2]:
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [3]:
import tensorflow as tf

2025-11-17 22:08:21.485619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-17 22:08:21.506332: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-17 22:08:21.512274: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-17 22:08:21.526329: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-17 22:08:22.449428: W tensorflow/compiler/tf2

In [4]:
def _decode_boxes(predicted_offsets: tf.Tensor, priors: tf.Tensor, variances: tf.Tensor):
    B = tf.shape(predicted_offsets)[0]
    N = tf.shape(predicted_offsets)[1]

    # Variance size
    variance_center = variances[0]
    variance_shape = variances[1]
    
    # Broadcasting the priors to the shape for the offsets
    broadcasted_priors = tf.broadcast_to(priors[tf.newaxis,...],[B,N,4])

    # Decoding the boxes
    tx,ty,tw,th = tf.split(predicted_offsets,num_or_size_splits = 4, axis=-1)
    cx,cy,w,h = tf.split(broadcasted_priors,num_or_size_splits = 4, axis=-1)

    # Adding the offset to the coordinates
    cx = cx + tx * variance_center * w
    cy = cy + ty * variance_center * h
    w = w * tf.math.exp(tw * variance_shape)
    h = h * tf.math.exp(th * variance_shape)

    # Converting the boxes to xy-coordinates
    x_min = cx - w/2
    y_min = cy - h/2
    x_max = cx + w/2
    y_max = cy + h/2

    boxes_xyxy = tf.concat([x_min, y_min, x_max, y_max],axis=-1)

    boxes_xyxy = tf.clip_by_value(boxes_xyxy,0,1)

    return boxes_xyxy

In [5]:
pred_loc = tf.constant(
    [[[0.0,  0.0,  0.0,  0.0],   # offsets for prior 0
      [0.5, -0.5,  0.3, -0.2]]], # offsets for prior 1
    dtype=tf.float32
)

priors = tf.constant(
    [[0.50, 0.50, 0.20, 0.40],   # (cx, cy, w, h) prior 0
     [0.25, 0.75, 0.10, 0.20]],  # prior 1
    dtype=tf.float32
)

variances = tf.constant([0.1, 0.2], dtype=tf.float32)

min_size = None

I0000 00:00:1763435303.236224   96181 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1763435303.324594   96181 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1763435303.324702   96181 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1763435303.326014   96181 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1763435303.326075   96181 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [6]:
pred_loc

<tf.Tensor: shape=(1, 2, 4), dtype=float32, numpy=
array([[[ 0. ,  0. ,  0. ,  0. ],
        [ 0.5, -0.5,  0.3, -0.2]]], dtype=float32)>

In [7]:
_decode_boxes(pred_loc,priors,variances)

<tf.Tensor: shape=(1, 2, 4), dtype=float32, numpy=
array([[[0.4       , 0.3       , 0.6       , 0.7       ],
        [0.20190817, 0.6439211 , 0.30809182, 0.83607894]]], dtype=float32)>

In [8]:
def _softmax_probabilities(logits: tf.Tensor):
    logits = tf.cast(logits, tf.float32)

    max_per_row = tf.reduce_max(logits, axis=-1, keepdims=True)
    shifted_logits = logits - max_per_row

    exponential_shifted_logits = tf.math.exp(shifted_logits)
    sum_exponential_shifted_logits = tf.reduce_sum(exponential_shifted_logits, axis=-1, keepdims=True)

    softmax_probs = exponential_shifted_logits / sum_exponential_shifted_logits

    return softmax_probs

def _sigmoid_probabilities(logits: tf.Tensor):
    logits = tf.cast(logits, tf.float32)

    sigmoid_probabilities = 1/(1 + tf.math.exp(-logits))

    return sigmoid_probabilities
    

def _score_from_logits(predicted_logits: tf.Tensor, scores_thresh: float, use_sigmoid: bool = True):
    if use_sigmoid:
        sigmoid_probs = _sigmoid_probabilities(predicted_logits)

        thresh_mask = sigmoid_probs < scores_thresh

        thresholded_scores = tf.where(thresh_mask, tf.zeros_like(sigmoid_probs), sigmoid_probs)
    else:
        softmax_probs = _softmax_probabilities(predicted_logits)
        scores = softmax_probs[...,1:]

        # Applying the threshold
        thresh_mask = scores < scores_thresh

        # Updating the scores in the tensor
        thresholded_scores = tf.where(thresh_mask, tf.zeros_like(scores), scores)
    
    return thresholded_scores

In [9]:
pred_logits = tf.constant([
    [
        [ 0.0,  2.0,  0.5, -1.0 ],
        [ 1.0, -0.5,  0.0,  0.0 ],
        [ 3.0,  3.0,  1.0,  0.0 ]
    ]
]
)

In [10]:
pred_logits

<tf.Tensor: shape=(1, 3, 4), dtype=float32, numpy=
array([[[ 0. ,  2. ,  0.5, -1. ],
        [ 1. , -0.5,  0. ,  0. ],
        [ 3. ,  3. ,  1. ,  0. ]]], dtype=float32)>

In [11]:
_score_from_logits(pred_logits, 0.1, use_sigmoid = False)

<tf.Tensor: shape=(1, 3, 3), dtype=float32, numpy=
array([[[0.7100999 , 0.1584447 , 0.        ],
        [0.11390647, 0.18780003, 0.18780003],
        [0.4576403 , 0.        , 0.        ]]], dtype=float32)>

In [12]:
pred_logits = tf.constant([
 [
     [ 0.0,  2.0, -1.0 ],
     [-0.5,  0.0,  1.0 ]
 ]   
]
)

In [13]:
pred_logits

<tf.Tensor: shape=(1, 2, 3), dtype=float32, numpy=
array([[[ 0. ,  2. , -1. ],
        [-0.5,  0. ,  1. ]]], dtype=float32)>

In [14]:
_sigmoid_probabilities(pred_logits)

<tf.Tensor: shape=(1, 2, 3), dtype=float32, numpy=
array([[[0.5       , 0.880797  , 0.26894143],
        [0.37754068, 0.5       , 0.73105854]]], dtype=float32)>

In [15]:
def _prepare_nms_inputs(boxes_xyxy: tf.Tensor, scores: tf.Tensor):
    nms_boxes = boxes_xyxy[:,:,tf.newaxis,:]
    nms_scores = scores
    
    return nms_boxes,nms_scores

In [16]:
boxes_xyxy = tf.constant([
    [  # image 0
        [0.10, 0.20, 0.40, 0.60],   # anchor 0
        [0.15, 0.25, 0.50, 0.70],   # anchor 1
        [0.60, 0.10, 0.90, 0.40],   # anchor 2
    ],
    [  # image 1
        [0.05, 0.30, 0.35, 0.80],   # anchor 0
        [0.20, 0.15, 0.55, 0.65],   # anchor 1
        [0.55, 0.50, 0.95, 0.95],   # anchor 2
    ],
])

scores = tf.constant([
    [  # image 0
        [0.80, 0.10],   # anchor 0: mostly class 0
        [0.60, 0.40],   # anchor 1: kind of ambiguous
        [0.05, 0.90],   # anchor 2: mostly class 1
    ],
    [  # image 1
        [0.20, 0.50],   # anchor 0
        [0.70, 0.30],   # anchor 1
        [0.10, 0.95],   # anchor 2
    ],
])

In [17]:
_prepare_nms_inputs(boxes_xyxy,scores)

(<tf.Tensor: shape=(2, 3, 1, 4), dtype=float32, numpy=
 array([[[[0.1 , 0.2 , 0.4 , 0.6 ]],
 
         [[0.15, 0.25, 0.5 , 0.7 ]],
 
         [[0.6 , 0.1 , 0.9 , 0.4 ]]],
 
 
        [[[0.05, 0.3 , 0.35, 0.8 ]],
 
         [[0.2 , 0.15, 0.55, 0.65]],
 
         [[0.55, 0.5 , 0.95, 0.95]]]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3, 2), dtype=float32, numpy=
 array([[[0.8 , 0.1 ],
         [0.6 , 0.4 ],
         [0.05, 0.9 ]],
 
        [[0.2 , 0.5 ],
         [0.7 , 0.3 ],
         [0.1 , 0.95]]], dtype=float32)>)

In [18]:
def _run_batched_nms(nms_boxes: tf.Tensor, nms_scores: tf.Tensor, iou_thresh: float, scores_thresh: float, top_k: int, max_detections: int):
    
    nms_boxes, nms_scores, nms_classes, valid_detections = tf.image.combined_non_max_suppression(nms_boxes,nms_scores,top_k,max_detections,iou_threshold= iou_thresh,score_threshold= scores_thresh,clip_boxes=False)

    return nms_boxes,nms_scores,nms_classes,valid_detections

In [19]:
nms_boxes_xyxy = tf.constant([
    [  # image 0
        [[0.10, 0.10, 0.40, 0.40]],   # anchor 0
        [[0.15, 0.15, 0.45, 0.45]],   # anchor 1
        [[0.60, 0.60, 0.90, 0.90]],   # anchor 2
    ],
])

scores = tf.constant([
    [  # image 0
        [0.90, 0.20],   # anchor 0
        [0.80, 0.70],   # anchor 1
        [0.10, 0.95],   # anchor 2
    ],
])

In [20]:
nms_boxes_xyxy

<tf.Tensor: shape=(1, 3, 1, 4), dtype=float32, numpy=
array([[[[0.1 , 0.1 , 0.4 , 0.4 ]],

        [[0.15, 0.15, 0.45, 0.45]],

        [[0.6 , 0.6 , 0.9 , 0.9 ]]]], dtype=float32)>

In [21]:
scores

<tf.Tensor: shape=(1, 3, 2), dtype=float32, numpy=
array([[[0.9 , 0.2 ],
        [0.8 , 0.7 ],
        [0.1 , 0.95]]], dtype=float32)>

In [22]:
_run_batched_nms(nms_boxes_xyxy,scores, 0.5, 0.5, 10, 3)

(<tf.Tensor: shape=(1, 3, 4), dtype=float32, numpy=
 array([[[0.6 , 0.6 , 0.9 , 0.9 ],
         [0.1 , 0.1 , 0.4 , 0.4 ],
         [0.15, 0.15, 0.45, 0.45]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.95, 0.9 , 0.7 ]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[1., 0., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>)

In [23]:
def _restore_to_image_space(boxes_yxyx_norm: tf.Tensor, image_height: int, image_width: int):
    # Multiplying the normalized coordinates with the 
    y_min,x_min,y_max,x_max = tf.split(boxes_yxyx_norm,num_or_size_splits = 4, axis=-1)

    # Scaling the box back to the image
    y_min = y_min * image_height
    x_min = x_min * image_width

    y_max = y_max * image_height
    x_max = x_max * image_width

    # Stacking the values back
    return tf.concat([x_min,y_min,x_max,y_max], axis=-1)


In [24]:
nmsed_boxes_norm = tf.constant([
    [
    [0.40, 0.30, 0.70, 0.60],   # box A
    [0.10, 0.55, 0.20, 0.85],   # box B
    ]
]
)

In [25]:
nmsed_boxes_norm

<tf.Tensor: shape=(1, 2, 4), dtype=float32, numpy=
array([[[0.4 , 0.3 , 0.7 , 0.6 ],
        [0.1 , 0.55, 0.2 , 0.85]]], dtype=float32)>

In [26]:
_restore_to_image_space(nmsed_boxes_norm,300,300)

<tf.Tensor: shape=(1, 2, 4), dtype=float32, numpy=
array([[[ 90., 120., 180., 210.],
        [165.,  30., 255.,  60.]]], dtype=float32)>

In [27]:
def _filter_small_boxes(boxes_norm_xyxy: tf.Tensor, scores:tf.Tensor, min_size: float):

    # Safety check
    if min_size is None or min_size <= 0:
         return boxes_norm_xyxy, scores, tf.ones(tf.shape(boxes_norm_xyxy)[:2], dtype=tf.bool)
    
    # Splitting the coordinates
    x_min,y_min,x_max,y_max = tf.split(boxes_norm_xyxy,num_or_size_splits = 4, axis=-1)

    # Calculating the width and height
    width = x_max - x_min
    height = y_max - y_min

    # Masking stuff to keep
    keep_mask = (width >= min_size) & (height >= min_size)

    # Gathering the box anchors
    gathered_anchors = tf.where(keep_mask, boxes_norm_xyxy, tf.zeros_like(boxes_norm_xyxy))

    # Gathering the scores
    gathered_scores = tf.where(keep_mask, scores, tf.zeros_like(scores))

    return gathered_anchors, gathered_scores, keep_mask

In [28]:
boxes_norm_xyxy = tf.constant([
  [  # image 0
    [0.10, 0.10, 0.50, 0.50],
    [0.20, 0.20, 0.25, 0.23],
    [0.00, 0.00, 0.15, 0.50],
    [0.80, 0.80, 0.90, 0.82],
  ],
  [  # image 1
    [0.05, 0.05, 0.07, 0.07],
    [0.30, 0.30, 0.70, 0.90],
    [0.50, 0.10, 0.90, 0.18],
    [0.00, 0.50, 0.10, 0.60],
  ],
])

scores = tf.constant(
    [
  [  # image 0
    [0.90, 0.10, 0.05],   # box 0
    [0.20, 0.30, 0.40],   # box 1
    [0.05, 0.80, 0.60],   # box 2
    [0.10, 0.20, 0.90],   # box 3
  ],
  [  # image 1
    [0.05, 0.10, 0.20],   # box 0
    [0.90, 0.70, 0.30],   # box 1
    [0.40, 0.50, 0.05],   # box 2
    [0.10, 0.80, 0.60],   # box 3
  ],
]
)

In [29]:
_filter_small_boxes(boxes_norm_xyxy,scores,0.1)

(<tf.Tensor: shape=(2, 4, 4), dtype=float32, numpy=
 array([[[0.1 , 0.1 , 0.5 , 0.5 ],
         [0.  , 0.  , 0.  , 0.  ],
         [0.  , 0.  , 0.15, 0.5 ],
         [0.  , 0.  , 0.  , 0.  ]],
 
        [[0.  , 0.  , 0.  , 0.  ],
         [0.3 , 0.3 , 0.7 , 0.9 ],
         [0.  , 0.  , 0.  , 0.  ],
         [0.  , 0.5 , 0.1 , 0.6 ]]], dtype=float32)>,
 <tf.Tensor: shape=(2, 4, 3), dtype=float32, numpy=
 array([[[0.9 , 0.1 , 0.05],
         [0.  , 0.  , 0.  ],
         [0.05, 0.8 , 0.6 ],
         [0.  , 0.  , 0.  ]],
 
        [[0.  , 0.  , 0.  ],
         [0.9 , 0.7 , 0.3 ],
         [0.  , 0.  , 0.  ],
         [0.1 , 0.8 , 0.6 ]]], dtype=float32)>,
 <tf.Tensor: shape=(2, 4, 1), dtype=bool, numpy=
 array([[[ True],
         [False],
         [ True],
         [False]],
 
        [[False],
         [ True],
         [False],
         [ True]]])>)

In [30]:
scores

<tf.Tensor: shape=(2, 4, 3), dtype=float32, numpy=
array([[[0.9 , 0.1 , 0.05],
        [0.2 , 0.3 , 0.4 ],
        [0.05, 0.8 , 0.6 ],
        [0.1 , 0.2 , 0.9 ]],

       [[0.05, 0.1 , 0.2 ],
        [0.9 , 0.7 , 0.3 ],
        [0.4 , 0.5 , 0.05],
        [0.1 , 0.8 , 0.6 ]]], dtype=float32)>

In [31]:
boxes_xyxy = tf.constant([
  [
    [0.10, 0.10, 0.20, 0.20],  # anchor 0
    [0.15, 0.15, 0.30, 0.30],  # anchor 1
    [0.40, 0.40, 0.60, 0.60],  # anchor 2
    [0.05, 0.50, 0.25, 0.70],  # anchor 3
    [0.60, 0.10, 0.90, 0.40],  # anchor 4
  ],
   [
    [0.05, 0.05, 0.15, 0.15],  # anchor 0
    [0.20, 0.20, 0.40, 0.40],  # anchor 1
    [0.45, 0.45, 0.65, 0.65],  # anchor 2
    [0.10, 0.60, 0.30, 0.80],  # anchor 3
    [0.55, 0.20, 0.85, 0.50],  # anchor 4
  ]
])

scores = tf.constant([
     [
    [0.90, 0.10, 0.05],  # anchor 0
    [0.20, 0.15, 0.10],  # anchor 1
    [0.50, 0.30, 0.25],  # anchor 2
    [0.80, 0.60, 0.40],  # anchor 3
    [0.10, 0.05, 0.02],  # anchor 4
  ],
    [
    [0.10, 0.05, 0.02],  # anchor 0
    [0.95, 0.80, 0.60],  # anchor 1
    [0.60, 0.55, 0.50],  # anchor 2
    [0.40, 0.20, 0.10],  # anchor 3
    [0.70, 0.65, 0.30],  # anchor 4
  ]
])

In [32]:
def _pre_nms_top_k(boxes_xyxy: tf.Tensor, scores: tf.Tensor, top_k: int):

    tf.debugging.assert_equal(tf.shape(boxes_xyxy)[-1], 4)
    tf.debugging.assert_equal(tf.shape(boxes_xyxy)[1], tf.shape(scores)[1])

    boxes_xyxy = tf.cast(boxes_xyxy,tf.float32)
    scores = tf.cast(scores,tf.float32)

    N = tf.shape(boxes_xyxy)[1]

    top_k = tf.minimum(N,top_k)

    # Max scores
    max_scores = tf.reduce_max(scores,axis=-1)

    # Get top k scores
    _, top_k_indices = tf.math.top_k(max_scores,k=top_k)

    # Get the top k boxes
    boxes_top_k = tf.gather(boxes_xyxy,top_k_indices, batch_dims = 1)

    # Gathering all the scores associated with the indices
    scores_top_k = tf.gather(scores,top_k_indices, batch_dims = 1)

    return boxes_top_k, scores_top_k

In [33]:
_pre_nms_top_k(boxes_xyxy,scores,3)

(<tf.Tensor: shape=(2, 3, 4), dtype=float32, numpy=
 array([[[0.1 , 0.1 , 0.2 , 0.2 ],
         [0.05, 0.5 , 0.25, 0.7 ],
         [0.4 , 0.4 , 0.6 , 0.6 ]],
 
        [[0.2 , 0.2 , 0.4 , 0.4 ],
         [0.55, 0.2 , 0.85, 0.5 ],
         [0.45, 0.45, 0.65, 0.65]]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3, 3), dtype=float32, numpy=
 array([[[0.9 , 0.1 , 0.05],
         [0.8 , 0.6 , 0.4 ],
         [0.5 , 0.3 , 0.25]],
 
        [[0.95, 0.8 , 0.6 ],
         [0.7 , 0.65, 0.3 ],
         [0.6 , 0.55, 0.5 ]]], dtype=float32)>)

In [34]:
def decode_and_nms(predicted_offsets: tf.Tensor, predicted_logits: tf.Tensor, priors: tf.Tensor, variances: tf.Tensor, scores_thresh: float, iou_thresh: float, top_k: int, max_detections: int, image_meta: dict| None, use_sigmoid: bool = False, **kwargs):
    # Decoding the boxes
    boxes_xyxy = _decode_boxes(predicted_offsets = predicted_offsets, priors = priors, variances = variances)

    # Calculating the scores from the logits
    scores = _score_from_logits(predicted_logits = predicted_logits, scores_thresh = scores_thresh, use_sigmoid = use_sigmoid)

    # Check if there is a filter option
    if 'min_box_size' in kwargs:
       boxes_xyxy, scores, keep_mask = _filter_small_boxes(boxes_xyxy, scores,kwargs['min_box_size'])

    if 'pre_nms_top_k' in kwargs:
        boxes_xyxy, scores = _pre_nms_top_k(boxes_xyxy, scores, kwargs['pre_nms_top_k'])

    # Preparing the inputs for NMS outputs
    nms_boxes, nms_scores = _prepare_nms_inputs(boxes_xyxy, scores)

    # Run batched NMS
    nmsed_boxes,nmsed_scores, nmsed_classes, valid_detections = _run_batched_nms(nms_boxes,nms_scores,iou_thresh, scores_thresh, top_k, max_detections)

    if not use_sigmoid:
        # Accounting for Softmax probs
        nmsed_classes = tf.cast(nmsed_classes,tf.int32)
        nmsed_classes = nmsed_classes + 1

    # Returning the boxes to image space
    if image_meta is not None:
        nmsed_boxes = _restore_to_image_space(nmsed_boxes,image_meta['image_height'], image_meta['image_width'])
    
    return nmsed_boxes,nmsed_scores, nmsed_classes, valid_detections

In [35]:
pred_loc = tf.constant(
    [
  [  # batch 0
    [ 0.0,  0.0,  0.0,  0.0],   # anchor 0
    [ 0.2,  0.0,  0.0,  0.0],   # anchor 1
    [ 0.0,  0.0,  0.5,  0.0],   # anchor 2
    [ 0.0, -0.2,  0.0, -0.5],   # anchor 3
  ]
]
)

pred_logits = tf.constant(
    [
  [  # batch 0
    [0.1,  2.0,  0.0],   # anchor 0
    [0.0,  0.5,  3.0],   # anchor 1
    [0.5,  1.5, -1.0],   # anchor 2
    [0.2, -0.5,  0.0],   # anchor 3
  ]
]
)

priors = tf.constant(
    [
  [0.25, 0.25, 0.2, 0.2],  # anchor 0 (top-left)
  [0.75, 0.25, 0.2, 0.2],  # anchor 1 (top-right)
  [0.25, 0.75, 0.2, 0.2],  # anchor 2 (bottom-left)
  [0.75, 0.75, 0.2, 0.2],  # anchor 3 (bottom-right)
]
)

variances      = tf.constant([0.1, 0.2])
score_thresh   = 0.3
iou_thresh     = 0.5
top_k          = 50           # big enough; not really limiting here
max_detections = 3
use_sigmoid    = False        # we’re using softmax with background
image_meta     = {'image_height': 300, 'image_width': 300}         # keep normalized coords for this example

In [36]:
decode_and_nms(pred_loc, pred_logits, priors, variances,score_thresh,iou_thresh,top_k,max_detections,image_meta,use_sigmoid)

(<tf.Tensor: shape=(1, 3, 4), dtype=float32, numpy=
 array([[[ 45.     , 196.2    , 105.     , 256.2    ],
         [ 45.     ,  45.     , 105.     , 105.     ],
         [195.     ,  41.84487, 255.     , 108.15513]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.88349205, 0.77826834, 0.68967205]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[2, 1, 1]], dtype=int32)>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>)